In [1]:
import sys
import h5py

import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing.image import apply_transform
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.ndimage as ndi
from sklearn.ensemble import RandomForestClassifier

sys.path.insert(0, r'C:\Users\fires\Desktop\CSIRO\thursday\thursday')

from format_data import add_random
from format_data import append_random
from format_data import augment
from format_data import generate_labels_fri
from format_data import generate_labels_frii
from format_data import get_aniyan
from format_data import get_fr
from format_data import get_random
from format_data import initial_resizing
from format_data import resize_array
from models import HOGNet
from models import SklearnModel

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Setting Seed
seed = 0

data_path = 'data/'
save_path = 'saved/'

# Setting Paths
raw_fr_data_data = data_path + 'data_raw.h5'
fr_data_path = data_path + 'data.h5'
random_path = data_path + 'random_1000.h5'
everything_path = data_path + 'everything.h5'

In [3]:
if not Path(raw_fr_data_data).is_file():
    raise print("Error: " + raw_fr_data_data + " is not a valid path and/or file")

if not Path(random_path).is_file():
    raise print("Error: " + random_path + " is not a valid path and/or file")

if not Path(fr_data_path).is_file():
    initial_resizing(raw_fr_data_data, fr_data_path)

if not Path(everything_path).is_file():
    add_random(fr_data_path, random_path, everything_path)

In [4]:
train_i, test_i = get_random(everything_path, split_ratio=(1/3), seed=0)

In [5]:
with h5py.File(everything_path, 'r') as data:
    images = np.asarray(data["images"])
    labels = np.asarray(data["labels"])
    
    train_x = images[train_i]
    train_x = resize_array(train_x, dim=256)
    train_x = np.expand_dims(train_x, axis=3)
    test_x = images[test_i]
    test_x = resize_array(test_x, dim=256)
    test_x = np.expand_dims(test_x, axis=3)
    
    train_y = np.where(labels[train_i]==1, True, False)
    test_y = np.where(labels[test_i]==1, True, False) 

In [6]:
test_x.shape

(416, 256, 256, 1)

In [7]:
datagen = augment(rotation_range=180, zoom_range=0.2, flip=True)

In [8]:
rft = SklearnModel(RandomForestClassifier, datagen, nb_augment=20, 
                   seed=0, n_estimators=10)

In [9]:
rft.fit(train_x, train_y)

In [10]:
rft.save(path=save_path+'rft.pk')

In [11]:
rft.load()
score = rft.predict_proba(test_x)

In [12]:
score

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [13]:
hognet = HOGNet(datagen=datagen, batch_size=50, steps_per_epoch=100, 
                max_epoch=5, patience=5, seed=seed)

C:\Users\fires\Desktop\CSIRO\thursday\thursday\models.py:440: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, kernel_size=(1, 1), strides=(1, 1), activation="relu", trainable=True, name="votes", use_bias=False)`
  trainable=True, bias=False, name="votes")(sin_cos_vec)


In [14]:
hognet.fit(train_x, train_y)

Epoch 1/5
100/100 [==============================] - 304s 3s/step - loss: 0.2612 - acc: 0.8954
Epoch 2/5
100/100 [==============================] - 485s 5s/step - loss: 0.1886 - acc: 0.9019
Epoch 3/5
100/100 [==============================] - 502s 5s/step - loss: 0.1849 - acc: 0.9011
Epoch 4/5
100/100 [==============================] - 498s 5s/step - loss: 0.1844 - acc: 0.9013
Epoch 5/5
100/100 [==============================] - 498s 5s/step - loss: 0.1837 - acc: 0.8992
Epoch 1/5
100/100 [==============================] - 485s 5s/step - loss: 0.1760 - acc: 0.9016
Epoch 2/5
100/100 [==============================] - 491s 5s/step - loss: 0.1641 - acc: 0.9124
Epoch 3/5
100/100 [==============================] - 476s 5s/step - loss: 0.1578 - acc: 0.9241
Epoch 4/5
100/100 [==============================] - 462s 5s/step - loss: 0.1499 - acc: 0.9254
Epoch 5/5
100/100 [==============================] - 451s 5s/step - loss: 0.1483 - acc: 0.9313


TypeError: 'LossHistory' object is not iterable

In [ ]:
hognet.save(path=save_path + "hognet.h5")

In [ ]:
hognet.load()

predictions = hognet.predict(test_x)

correct = test_y == predictions
print('Accuracy: {:.02%}'.format(correct.mean()))
print (predictions)